# Aim

Get FFAST working at a basic level (no noise, no window_var, etc.).

- for reasonably high-snr like around 5db (without the bin-length gain) look at kay vs. kay2 my guess is kay will fail but kay2 will keep working.  how much db can you go lower with kay2. is kay2 alway better?
- how does all this compare to "new" method?
- keep an eye on the total number of samples used by each method.

In [4]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from ffast import *
from matplotlib import pyplot as plt
import numpy as np
import time

from tqdm import tqdm_notebook as tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
params = make_args()

params.primes = [2, 3]
params.prime_powers = [7, 5]

params.bins = [i**j for i,j in zip(params.primes, params.prime_powers)]
print(params.bins)

params.length = np.prod(params.bins)
print(np.log2(params.length))

params.iterations = 50
params.snr = 10#*np.log10(max(params.bins))

params.sparsity = 1
params.bin_processing_method = ['ml', 'kay', 'kay2', 'new'][2]
chains_and_delays = {'ml': (120, 1), 'kay': (14, 5), 'kay2': (14, 2), 'new': (1, 8)}
params.chains, params.delays = chains_and_delays[params.bin_processing_method]

config = Config(params)
config.compute_params()

input_signal = ExperimentInputSignal(config)
input_signal.process()
output_signal = ExperimentOutputSignal(config, input_signal)

In [ ]:
config.get_noise_sd()**2

In [ ]:
# vars(config)

In [ ]:
config.display()

In [ ]:
plt.plot(np.real(input_signal.time_signal))
plt.plot(np.imag(input_signal.time_signal))
plt.show()

In [ ]:
# len(input_signal.time_signal)

In [ ]:
signal_fft = np.fft.fft(input_signal.time_signal)
plt.plot(np.abs(signal_fft))
plt.title('loc: {}'.format(input_signal.freqs))
plt.show()

In [ ]:
# input_signal.time_signal *= 2 # demonstrating that the input signal can be changed

In [ ]:
ffast = FFAST(config, input_signal, output_signal)

In [ ]:
ffast.frontend.process()
print(ffast.frontend.get_used_samples_nb())

In [ ]:
params.bins

In [ ]:
ffast = FFAST(config, input_signal, output_signal)

if not config.help_displayed:
    # this disregards the time to make the signal
    time_initial = int(np.round(time.time() * 1000)) 

    iterations = 3 or config.iterations
    
    for i in tqdm(range(1)):
        input_signal.process() # ffast.get_delays() in the CPP code as an argument
        print('input freq: {}'.format(input_signal.freqs))
        
        print('true singleton bins:')
        for stage_idx, bin_size in enumerate(params.bins):
            print('    stage {} bin {}'.format(stage_idx, input_signal.freqs[0] % bin_size))
        
        ffast.process()
        output_signal.process()
        print('---')

    time_final = int(np.round(time.time() * 1000))

    ffast.display_results(time_final - time_initial)

In [ ]:
output_signal.statistics()[0]/iterations

# this part is for debugging

In [ ]:
vars(config)

In [ ]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print("true frequencies: {}".format(input_signal.freqs))

ffast.frontend.process()
# ffast.backend.process()
# output_signal.process()

In [ ]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.xlabel('bin absolute index')
# plt.show()

In [ ]:
singleton_bin = input_signal.freqs[0] % params.bins[0]
print(singleton_bin)

In [ ]:
binprocessor = BinProcessor(config, ffast.frontend.delays, ffast.frontend.observation_matrix)

In [ ]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.axhline(binprocessor.thresholds[0])
# plt.xlabel('bin absolute index')
# plt.show()

In [ ]:
binprocessor.adjust_to(singleton_bin, singleton_bin, 0)

In [ ]:
binprocessor.is_zeroton()

In [ ]:
binprocessor.process()

In [ ]:
# binprocessor.find_location_new()
print(binprocessor.location)
print(input_signal.freqs)

In [ ]:
binprocessor.is_singleton()

In [ ]:
binprocessor.location

In [ ]:
binprocessor.amplitude

In [ ]:
binprocessor.noise

In [ ]:
binprocessor.thresholds[0]

In [ ]:
binprocessor.minimum_energy

In [ ]:
ffast.backend.decoded_frequencies

In [ ]:
output_signal.check_full_recovery()

In [ ]:
output_signal.statistics()

In [ ]:
ffast = FFAST(config, input_signal, output_signal)

In [ ]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print(input_signal.freqs)

In [ ]:
ffast.process()

In [ ]:
output_signal.process()

In [ ]:
output_signal.statistics()

In [ ]:
ffast.display_results(0)